## Colab Setup

In [ ]:
# if you run this notebook in kaggle notebook or other platform, comment out the following codef
from google.colab import drive
drive.mount('/content/drive')

## Config

In [ ]:
root = '/content/drive/MyDrive/Colab Notebooks/g2net/' # set your root directory in your google drive. if you use Kaggle notebook, set this to '.'
OUTPUT_DIR = root+'/test-B7/' # set your current folder to save model weights and outputs files
DATA_DIR = root + '/data/' # put whiten profile data here to run whiten process on the fly
MODEL_DIR = OUTPUT_DIR + '/models/' # put model weights to estimate here

In [ ]:
import numpy as np

# CQT params

#FMIN=22 # 20
#FMAX=None
#WINDOW_TYPE='nuttall'
#BINS=64
HOP_LENGTH = 32
#SCALE=1
NORM=1
#OCTAVE=12

FMIN=20 # 20
FMAX=512
BINS=84
OCTAVE=24
SCALE=0.4
WINDOW_TYPE="hann"

SMOOTHING=0.00
ST=int(4096 / 16 * 7)
EN=int(4096 / 16 * 15)

NUM_FOLDS = 5
FOLDS=[0, 1, 2, 3, 4] 

LR=1e-4 # 1e-4
IMAGE_SIZE = 512 #
BATCH_SIZE = 16 # 32
EFFICIENTNET_SIZE = 7
WEIGHTS =  "imagenet" #"noisy-student"#"imagenet"

#NORMALIZE=True

MIXED=False # mixed precision does not work with tf models
TFHUB_MODEL=None #'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5' # 0876

MIXUP_PROB = 0.0
EPOCHS = 20
R_ANGLE = 0 / 180 * np.pi
S_SHIFT = 0.0
T_SHIFT = 0.0
LABEL_POSITIVE_SHIFT = 1.0

SEED = 2021

# https://www.kaggle.com/yamsam/g2net-tf-on-the-fly-cqt-tpu-inference-path
FILES =[
# 'gs://kds-f56f84a6d403c2466d12eed4d4afaa1fe1464a9723336d38f33ca366',
# 'gs://kds-d482711d73bef82b2ca8c1a0bd869a564992cdd7e6997df7b372ce8e'

'gs://kds-9555cba0c858cb42c22a7a077204092f9007dff650116a7d4c173091',
'gs://kds-02cc6d7b70e4ba9c2e7f31c15fb2dd7455ce6ad21c7afc2e08ada695'
]


In [ ]:
!pip install efficientnet tensorflow_addons > /dev/null

In [ ]:
import os
import math
import random
import re
import warnings
from pathlib import Path
from typing import Optional, Tuple

import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
#from kaggle_datasets import KaggleDatasets
from scipy.signal import get_window
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from tensorflow.keras import mixed_precision
import tensorflow_hub as hub

In [ ]:
tf.__version__

## Utilities

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


set_seed(SEED)

In [ ]:
def auto_select_accelerator():
    TPU_DETECTED = False
    try:
        if MIXED and TFHUB_MODEL is None:
          tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
          tf.config.experimental_connect_to_cluster(tpu)
          tf.tpu.experimental.initialize_tpu_system(tpu)
          strategy = tf.distribute.experimental.TPUStrategy(tpu)
          policy = mixed_precision.Policy('mixed_bfloat16')
          mixed_precision.set_global_policy(policy)
          tf.config.optimizer.set_jit(True)
        else:
          tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
          tf.config.experimental_connect_to_cluster(tpu)
          tf.tpu.experimental.initialize_tpu_system(tpu)
          strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
        TPU_DETECTED = True
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy, TPU_DETECTED

In [ ]:
strategy, tpu_detected = auto_select_accelerator()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

## Data Loading

In [ ]:
gcs_paths = []
for file in FILES:
    gcs_paths.append(file)
    print(file)

In [ ]:
all_files = []

for path in gcs_paths:
    all_files.extend(np.sort(np.array(tf.io.gfile.glob(path + f"/*.tfrecords")))) # !!!

print('test_files: ', len(all_files))

## Dataset Preparation

Here's the main contribution of this notebook - Tensorflow version of on-the-fly CQT computation. Note that some of the operations used in CQT computation are not supported by TPU, therefore the implementation is not a TF layer but a function that runs on CPU.

In [ ]:
def create_cqt_kernels(
    q: float,
    fs: float,
    fmin: float,
    n_bins: int = 84,
    bins_per_octave: int = 12,
    norm: float = 1,
    window: str = "hann",
    fmax: Optional[float] = None,
    topbin_check: bool = True
) -> Tuple[np.ndarray, int, np.ndarray, float]:
    fft_len = 2 ** _nextpow2(np.ceil(q * fs / fmin))
    
    if (fmax is not None) and (n_bins is None):
        n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
    elif (fmax is None) and (n_bins is not None):
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
    else:
        warnings.warn("If nmax is given, n_bins will be ignored", SyntaxWarning)
        n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
        
    if np.max(freqs) > fs / 2 and topbin_check:
        raise ValueError(f"The top bin {np.max(freqs)} Hz has exceeded the Nyquist frequency, \
                           please reduce the `n_bins`")
    
    kernel = np.zeros((int(n_bins), int(fft_len)), dtype=np.complex64)
    
    length = np.ceil(q * fs / freqs)
    for k in range(0, int(n_bins)):
        freq = freqs[k]
        l = np.ceil(q * fs / freq)
        
        if l % 2 == 1:
            start = int(np.ceil(fft_len / 2.0 - l / 2.0)) - 1
        else:
            start = int(np.ceil(fft_len / 2.0 - l / 2.0))

        sig = get_window(window, int(l), fftbins=True) * np.exp(
            np.r_[-l // 2:l // 2] * 1j * 2 * np.pi * freq / fs) / l
        
        if norm:
            kernel[k, start:start + int(l)] = sig / np.linalg.norm(sig, norm)
        else:
            kernel[k, start:start + int(l)] = sig
    return kernel, fft_len, length, freqs


def _nextpow2(a: float) -> int:
    return int(np.ceil(np.log2(a)))


def prepare_cqt_kernel(
    sr=22050,
    hop_length=512,
    fmin=32.70,
    fmax=None,
    n_bins=84,
    bins_per_octave=12,
    norm=1,
    filter_scale=1,
    window="hann"
):
    q = float(filter_scale) / (2 ** (1 / bins_per_octave) - 1)
    print(q)
    return create_cqt_kernels(q, sr, fmin, n_bins, bins_per_octave, norm, window, fmax)

In [ ]:

cqt_kernels, KERNEL_WIDTH, lengths, _ = prepare_cqt_kernel(
    sr=2048,
    hop_length=HOP_LENGTH,
    fmin=FMIN,
    fmax=FMAX,
    n_bins=BINS,
    norm=NORM,
    window=WINDOW_TYPE,
    bins_per_octave=OCTAVE,
    filter_scale=SCALE)
LENGTHS = tf.constant(lengths, dtype=tf.float32)
CQT_KERNELS_REAL = tf.constant(np.swapaxes(cqt_kernels.real[:, np.newaxis, :], 0, 2))
CQT_KERNELS_IMAG = tf.constant(np.swapaxes(cqt_kernels.imag[:, np.newaxis, :], 0, 2))
PADDING = tf.constant([[0, 0],
                        [KERNEL_WIDTH // 2, KERNEL_WIDTH // 2],
                        [0, 0]])

In [ ]:
ORDER=[0,0,0]
def create_cqt_image(wave, hop_length=16):
    CQTs = []
    for i in ORDER:
        x = wave[i][ST:EN]
        x = tf.expand_dims(tf.expand_dims(x, 0), 2)
        x = tf.pad(x, PADDING, "REFLECT")

        CQT_real = tf.nn.conv1d(x, CQT_KERNELS_REAL, stride=hop_length, padding="VALID")
        CQT_imag = -tf.nn.conv1d(x, CQT_KERNELS_IMAG, stride=hop_length, padding="VALID")
        CQT_real *= tf.math.sqrt(LENGTHS)
        CQT_imag *= tf.math.sqrt(LENGTHS)

        CQT = tf.math.sqrt(tf.pow(CQT_real, 2) + tf.pow(CQT_imag, 2))
        CQTs.append(CQT[0])
    return tf.stack(CQTs, axis=2)

In [ ]:
def read_labeled_tfrecord(example):
    tfrec_format = {
        "wave": tf.io.FixedLenFeature([], tf.string),
        "wave_id": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return prepare_image(example["wave"], IMAGE_SIZE), tf.reshape(tf.cast(example["target"], tf.float32), [1])


def read_unlabeled_tfrecord(example, return_image_id):
    tfrec_format = {
        "wave": tf.io.FixedLenFeature([], tf.string),
        "wave_id": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return prepare_image(example["wave"], IMAGE_SIZE), example["wave_id"] if return_image_id else 0


def count_data_items(filenames): 
    # The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

def mixup(image, label, probability=0.5, aug_batch=64 * 8):
    imgs = []
    labs = []
    for j in range(aug_batch):
        p = tf.cast(tf.random.uniform([], 0, 1) <= probability, tf.float32)
        k = tf.cast(tf.random.uniform([], 0, aug_batch), tf.int32)
        a = tf.random.uniform([], 0, 1) * p

        img1 = image[j]
        img2 = image[k]
        imgs.append((1 - a) * img1 + a * img2)
        lab1 = label[j]
        lab2 = label[k]
        labs.append((1 - a) * lab1 + a * lab2)
    image2 = tf.reshape(tf.stack(imgs), (aug_batch, IMAGE_SIZE, IMAGE_SIZE, 3))
    label2 = tf.reshape(tf.stack(labs), (aug_batch,))
    return image2, label2


def time_shift(img, shift=T_SHIFT):
    if shift > 0:
        T = IMAGE_SIZE
        P = tf.random.uniform([],0,1)
        SHIFT = tf.cast(T * P, tf.int32)
        return tf.concat([img[-SHIFT:], img[:-SHIFT]], axis=0)
    return img


def rotate(img, angle=R_ANGLE):
    if angle > 0:
        P = tf.random.uniform([],0,1)
        A = tf.cast(angle * P, tf.float32)
        return tfa.image.rotate(img, A)
    return img


def spector_shift(img, shift=S_SHIFT):
    if shift > 0:
        T = IMAGE_SIZE
        P = tf.random.uniform([],0,1)
        SHIFT = tf.cast(T * P, tf.int32)
        return tf.concat([img[:, -SHIFT:], img[:, :-SHIFT]], axis=1)
    return img

def img_aug_f(img):
#    img = time_shift(img)
#    img = spector_shift(img)
    #img = tf.image.random_flip_left_right(img) 
#    img = tf.image.random_brightness(img, 0.2)
#    img = AUGMENTATIONS_TRAIN(image=img)['image']
    # img = rotate(img)
    #print(img.shape)
    img = swap_img(img)
    return img


def swap_img(img):
   p = tf.random.uniform([],0,1)
   if p < 0.2:
     img = tf.stack([img[:,:,1], img[:,:,0], img[:,:,2]],axis=2)
     return  img
   else:
     return img


def imgs_aug_f(imgs, batch_size):
    _imgs = []
    DIM = IMAGE_SIZE
    for j in range(batch_size):
        _imgs.append(img_aug_f(imgs[j]))

    return tf.reshape(tf.stack(_imgs),(batch_size,DIM,DIM,3))


def label_positive_shift(labels):
    return labels * LABEL_POSITIVE_SHIFT


def aug_f(imgs, labels, batch_size):
    #imgs, label = mixup(imgs, labels, MIXUP_PROB, batch_size)
    imgs = imgs_aug_f(imgs, batch_size) 
    return imgs, labels

# used for whitening
window = tf.cast(np.load(DATA_DIR+'window.npy'), tf.float64)
arv_w = tf.cast(np.load(DATA_DIR+'avr_w.npy'), tf.complex64)

def whiten(c):
  #print (c.shape)
  c2 = tf.concat([tf.reverse(-c, axis=[1])[:,4096-2049:-1] + 2 *c[:,:1], c, tf.reverse(-c, axis=[1])[:,1:2049] + 2*c[:,-2:-1]],axis=1)
  #print (c2.shape)
  c3 = tf.math.real(tf.signal.ifft(tf.signal.fft(tf.cast(1e20*c2*window, tf.complex64))/arv_w))[:,2048:-2048]
  #print (c3.shape)
  return c3


def prepare_image(wave, dim=256):
    wave = tf.reshape(tf.io.decode_raw(wave, tf.float64), (3, 4096))
    #wave = tf.cast(wave, tf.float32)
    wave = whiten(wave)
    # normalized_waves = []
    # for i in range(3):
    #     normalized_wave = wave[i] - means[i]
    #     normalized_wave = normalized_wave / stds[i]

    #     normalized_waves.append(normalized_wave)
    # wave = tf.stack(normalized_waves)
    wave = tf.cast(wave, tf.float32)
    image = create_cqt_image(wave, HOP_LENGTH)
    #image = tf.keras.layers.Normalization()(image)
    image = tf.image.resize(image, size=(dim, dim))
    return tf.reshape(image, (dim, dim, 3))


def get_dataset(files, batch_size=16, repeat=False, shuffle=False, aug=True, labeled=True, return_image_ids=True):
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO, compression_type="GZIP")
    ds = ds.cache()

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(1024 * 2)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)

    if labeled:
        ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: read_unlabeled_tfrecord(example, return_image_ids), num_parallel_calls=AUTO)

    ds = ds.batch(batch_size * REPLICAS)
    if aug:
        ds = ds.map(lambda x, y: aug_f(x, y, batch_size * REPLICAS), num_parallel_calls=AUTO)
    ds = ds.prefetch(AUTO)
    return ds

## Model

In [ ]:
def build_model(size=256, efficientnet_size=0, weights="imagenet", count=0):
    inputs = tf.keras.layers.Input(shape=(size, size, 3))
    
    if TFHUB_MODEL:
      print ('using tf hubmodels')
      load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
      loaded_model = hub.load(TFHUB_MODEL, options=load_options)
      efn_layer = hub.KerasLayer(loaded_model, trainable=True) 

      x = efn_layer(inputs)
    else:
      efn_string= f"EfficientNetB{efficientnet_size}"
      efn_layer = getattr(efn, efn_string)(input_shape=(size, size, 3), weights=weights, include_top=False)
      x = efn_layer(inputs)
      x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    lr_decayed_fn = tf.keras.experimental.CosineDecay(1e-3, count)
    opt = tfa.optimizers.AdamW(lr_decayed_fn, learning_rate=LR)
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=SMOOTHING)
    #loss = tfa.losses.SigmoidFocalCrossEntropy()

    model.compile(optimizer=opt, loss=loss, metrics=["AUC"])
    return model

In [ ]:
def get_lr_callback(batch_size=8, replicas=8):
    lr_start   = 1e-4
    lr_max     = 0.000015 * replicas * batch_size
    lr_min     = 1e-7
    lr_ramp_ep = 3
    lr_sus_ep  = 0
    lr_decay   = 0.7
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)
    return lr_callback

In [ ]:
#plot
def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    # for i in range(3):
    #   image[i,:] -= image[i,:].min()
    #   image[i,:] /= image[i,:].max()
#    print (image.shape)
    plt.imshow(image[:,:,0].transpose())
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1

def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def display_9_images_from_dataset(dataset):
    subplot=331
    plt.figure(figsize=(13,13))
    images, labels = dataset_to_numpy_util(dataset, 9)
    for i, image in enumerate(images):
        title = labels[i]
        subplot = display_one_flower(image, f'{title}', subplot)
        if i >= 8:

            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()  

In [ ]:
ds = get_dataset(all_files[0], labeled=False, return_image_ids=False, repeat=False, shuffle=True, batch_size=BATCH_SIZE * 2, aug=False)
display_9_images_from_dataset(ds)

In [ ]:
#display_9_images_from_dataset(ds)

## Inference

In [ ]:
files_test_all = np.array(all_files)
all_test_preds = []

In [ ]:
# def count_data_items(filenames): 
#     # The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
#     n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
#     return np.sum(n)

# count_data_items_test = count_data_items

# for arais dataset
# def count_data_items(fileids):
#     return len(fileids) * 5600 # 28000

def count_data_items_test(fileids):
    return len(fileids) * 22600

print (count_data_items_test(files_test_all))

In [ ]:
with strategy.scope():
    model = build_model(
        size=IMAGE_SIZE,
        efficientnet_size=EFFICIENTNET_SIZE,
        weights=WEIGHTS,
        count=0)


In [ ]:
ORDER=[0,1,2]

for i in FOLDS:
    print(f"Load weight for Fold {i + 1} model")
    model.load_weights(MODEL_DIR + f"fold{i}.h5")
    
    ds_test = get_dataset(files_test_all, batch_size=BATCH_SIZE * 2, repeat=True, shuffle=False, aug=False, labeled=False, return_image_ids=False)
    STEPS = count_data_items_test(files_test_all) / BATCH_SIZE / 2 / REPLICAS
    pred = model.predict(ds_test, verbose=1, steps=STEPS)[:count_data_items_test(files_test_all)]

    all_test_preds.append(pred.reshape(-1))

In [ ]:
%%time
ORDER=[1,0,2]

for i in FOLDS:
    print(f"Load weight for Fold {i + 1} model")
    model.load_weights(MODEL_DIR + f"fold{i}.h5")
    
    ds_test = get_dataset(files_test_all, batch_size=BATCH_SIZE * 2, repeat=True, shuffle=False, aug=False, labeled=False, return_image_ids=False)
    STEPS = count_data_items_test(files_test_all) / BATCH_SIZE / 2 / REPLICAS
    pred = model.predict(ds_test, verbose=1, steps=STEPS)[:count_data_items_test(files_test_all)]

    all_test_preds.append(pred.reshape(-1))

In [ ]:
ds_test = get_dataset(files_test_all, batch_size=BATCH_SIZE * 2, repeat=False, shuffle=False, aug=False, labeled=False, return_image_ids=True)
file_ids = np.array([target.numpy() for img, target in iter(ds_test.unbatch())])

In [ ]:
test_pred = np.zeros_like(all_test_preds[0])
for i in range(len(all_test_preds)):
    test_pred += all_test_preds[i] / len(all_test_preds)
    
test_df = pd.DataFrame({
    "id": [i.decode("UTF-8") for i in file_ids],
    "target": test_pred.astype(float)
})

test_df.head()

In [ ]:
test_df.to_csv(OUTPUT_DIR + "submission.csv", index=False)

In [ ]:
import seaborn as sns
sns.distplot(test_df.target)

In [ ]:
for i in range(len(all_test_preds)):
  test_df[f'pred{i}'] = all_test_preds[i].astype(float)

test_df.to_csv(OUTPUT_DIR + "all_pred_tta_float.csv", index=False)
test_df.head()

In [ ]:
test_df.head()